In [1]:

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score

from qiskit import *
from qiskit.circuit import Parameter
from qiskit import IBMQ, transpile
from qiskit.providers.ibmq.managed import IBMQJobManager

import numpy as np

import pickle

In [2]:
def scale_problems(problems, scale_min=-np.pi/4, scale_max=np.pi/4):
    scaled = []
    scaler = MinMaxScaler((-np.pi/4, np.pi/4))
    for entry in problems:
        scaled.append(np.array(scaler.fit_transform(entry.reshape(-1, 1)).flatten()))
    return scaled

In [17]:
def uncertainity_principle(circuit):
    circuit.h(range(circuit.width()))
    circuit.barrier()

def cost_encoding(circuit):
    for i in range(circuit.width()):
        circuit.ry(-Parameter('c'+str(i)), i)
    circuit.barrier()

def same_query_cost(circuit):
    for i in range(0,circuit.width(),2):
        circuit.crz(-np.pi/4,i,i+1)
    circuit.barrier()

def savings_encoding(circuit):
        for i in range(int(circuit.width()/2)):
            circuit.crz(Parameter('s'+str(i)+str(int(circuit.width()/2))), i, int(circuit.width()/2))
            circuit.crz(Parameter('s'+str(i)+str(int(1+circuit.width()/2))), i, int(1+circuit.width()/2))
        circuit.barrier()

def rx_layer(circuit, weight):
    circuit.rx(weight, range(circuit.width()))
    circuit.barrier()

def create_circuit(n_queries=2, n_plans=2, scheme="hcsx", xweight=np.pi/4):
    circuit = QuantumCircuit(n_queries*n_plans)
    for module in scheme:
        if module == "h":
            uncertainity_principle(circuit)
        elif module == "c":
            cost_encoding(circuit)
        elif module == "s":
            savings_encoding(circuit)
        elif module == "x":
            rx_layer(circuit, xweight)
    return circuit

In [18]:
def parse_results(result):
    remove_useless_keys(result)
    for i, key in enumerate(["0101", "1001", "0110", "1010"]):
        result[i] = result.pop(key, 0)
    return max(result, key=result.get)

def remove_useless_keys(result):
    for key in ["0000","0001","0010","0100","1000","1101","1011","1100","0011","0111","1110","1111"]:
            if key in result:
                del result[key]
    for key in ["0101", "1001", "0110", "1010"]:
        if not key in result:
            result[key] = 0

In [5]:
def run_circuit(circuit, backend=Aer.get_backend("aer_simulator"), shots=1024):
    job = backend.run(transpile(circuit, backend), shots=shots)
    res = job.result()
    return res.get_counts(qc)

In [6]:
def score_results(results, solutions):
    return accuracy_score(results, solutions)

In [7]:
ibm_backend = 'ibmq_bogota'
total_iterations = 13

In [8]:
path = "data/problems_with_solutions.p"
data = pickle.load(open(path, "rb"))

In [9]:
x_train = scale_problems(data['x_train'])
x_test = scale_problems(data['x_test'])
y_train = data['y_train']
y_test = data['y_test']

In [19]:
circuit = create_circuit()
print(circuit.draw())

     ┌───┐ ░ ┌─────────────┐ ░                                              ░ »
q_0: ┤ H ├─░─┤ Ry(-1.0*c0) ├─░──────■──────────■────────────────────────────░─»
     ├───┤ ░ ├─────────────┤ ░      │          │                            ░ »
q_1: ┤ H ├─░─┤ Ry(-1.0*c1) ├─░──────┼──────────┼──────────■──────────■──────░─»
     ├───┤ ░ ├─────────────┤ ░ ┌────┴────┐     │     ┌────┴────┐     │      ░ »
q_2: ┤ H ├─░─┤ Ry(-1.0*c2) ├─░─┤ Rz(s02) ├─────┼─────┤ Rz(s12) ├─────┼──────░─»
     ├───┤ ░ ├─────────────┤ ░ └─────────┘┌────┴────┐└─────────┘┌────┴────┐ ░ »
q_3: ┤ H ├─░─┤ Ry(-1.0*c3) ├─░────────────┤ Rz(s03) ├───────────┤ Rz(s13) ├─░─»
     └───┘ ░ └─────────────┘ ░            └─────────┘           └─────────┘ ░ »
«     ┌─────────┐ ░ 
«q_0: ┤ Rx(π/4) ├─░─
«     ├─────────┤ ░ 
«q_1: ┤ Rx(π/4) ├─░─
«     ├─────────┤ ░ 
«q_2: ┤ Rx(π/4) ├─░─
«     ├─────────┤ ░ 
«q_3: ┤ Rx(π/4) ├─░─
«     └─────────┘ ░ 


In [11]:
#prepare circuits
circuits = []
for test in x_test:
    qc = circuit.copy().bind_parameters(test)
    qc.measure_all()
    circuits.append(qc)

In [15]:
simulation_accuracies = []
for i in range(total_iterations):
    results = []
    for qc in circuits:
        counts = run_circuit(qc.copy(), shots=4000)
        results.append(parse_results(counts))
    simulation_accuracies.append(score_results(results, y_test))

/home/ricardo/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/metrics/_classification.py:217: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/home/ricardo/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/metrics/_classification.py:217: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/home/ricardo/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/metrics/_classification.py:217: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  score = y_true == y_pred
/home/ricardo/miniconda3/envs/thesis/lib/python3.9/site-packages/sklearn/metrics/_classification.py:217: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise compar

In [100]:
provider = IBMQ.load_account()
backend = provider.get_backend(ibm_backend)

ibmqfactory.load_account:WARNING:2022-05-11 20:38:35,487: Credentials are already in use. The existing account in the session will be replaced.


In [51]:
transpiled_circuits = []
for qc in circuits:
    transpiled_circuits.append(transpile(qc, backend=backend))

In [52]:
job_manager = IBMQJobManager()

In [61]:
real_accuracies = []

In [135]:
for i in range(total_iterations):
    print('Iteration ' + str(i+1))
    job_set_static = job_manager.run(transpiled_circuits, backend=backend, name='Static MQO Solver Iteration ' + str(i))
    real_results = job_set_static.results()
    res_temp = []
    for j in range(len(transpiled_circuits)):
        res_temp.append(parse_results(real_results.get_counts(j)))
    real_accuracies.append(score_results(res_temp, y_test))

Iteration 1
Iteration 2
Iteration 3
Iteration 4
Iteration 5
Iteration 6
Iteration 7
Iteration 8
Iteration 9
Iteration 10
Iteration 11
Iteration 12
Iteration 13


In [20]:
path = 'experiments/final_runs/'
savename = 'static_solver.p'
try:
    os.makedirs(path)
except:
    print('path already exists!')

path already exists!


In [ ]:

provider = IBMQ.load_account()
backend = provider.get_backend(ibm_backend)
job_manager = IBMQJobManager()

In [69]:
ids =  ['627c0b7b2a75604410d0c09f', '627c0cd02a75608132d0c0ac', '627c0d702a75607a52d0c0b3', 
        '627c0e92530a86ddd9b2265f', '627c0fbf1cd6f5917e461656', '627c10e0b723351af8d95107', 
        '627c1179646e0502b09854f7', '627c1298ea4719ad7fd5cdce', '627c13ba21decb04d88656f6', 
        '627c14dd1cd6f54c3d461683', '627c161317f5c8069c1a1e27', '627c40ca17f5c8ae621a1f0d',
        '627c41497e6d3a64f8e549a7']
real_results = []
for id in ids:
        real_results.append(backend.retrieve_job(id).result().get_counts())
        

In [74]:
real_accuracies = []
for i, job in enumerate(real_results):
    res_temp = []
    for result in job:
        res_temp.append(parse_results(result))
    real_accuracies.append(score_results(res_temp, y_test))
    

In [76]:
simulation_accuracies

[0.8666666666666667,
 0.8666666666666667,
 0.8533333333333334,
 0.8666666666666667,
 0.8666666666666667,
 0.8666666666666667,
 0.8933333333333333,
 0.88,
 0.8933333333333333,
 0.8933333333333333,
 0.8666666666666667,
 0.8933333333333333,
 0.84]

In [77]:
run_data = {'accuracy_simulator':simulation_accuracies, 'accuracy_real_hardware': real_accuracies}

In [23]:

pickle.dump(run_data, open( path+savename, "wb" ) )    